# Using GLM-4.5V for Video Understanding

[GLM-4.5V](https://docs.bigmodel.cn/cn/guide/models/vlm/glm-4.5v) is Z.AI's new generation visual reasoning model based on MOE architecture. With 106B total parameters and 12B activation parameters, it achieves SOTA performance among global open-source multimodal models of the same level across various benchmarks, covering common tasks such as image, video, document understanding, and GUI tasks. This code demonstrates how to use GLM-4.5V's video understanding functionality with videos.


First, set the API key for calling the model

In [3]:
import os
from zhipuai import ZhipuAI
os.environ["ZHIPUAI_API_KEY"] = "your api key"

client = ZhipuAI()


Next, input the video into the model. Please note that only the `GLM-4.5V` model supports video understanding. After that, we need to encode the video into video_base64. The analysis can then be performed by the large model. A video understanding process usually takes only a few seconds. We can see the answer very quickly.

In [4]:
import base64

video_path = "data/video_1.mp4"
with open(video_path, 'rb') as video_file:
    video_base = base64.b64encode(video_file.read()).decode('utf-8')

response = client.chat.completions.create(
    model="glm-4.5v",
    temperature=0.0,
    top_p=0,
    messages=[
      {
        "role": "user",
        "content": [
          {
            "type": "video_url",
            "video_url": {
                "url" : video_base
            }
          },
          {
            "type": "text",
            "text": "Please carefully describe the environment in this video, what the little dog is doing, and tell me in an interesting way"
          }
        ]
      }
    ]
)
print(response.choices[0].message.content)


<|begin_of_box|>The scene is set in a **serene, rain-kissed body of water**—likely a quiet pond or lazy stream—where the greenish hue of the water glows under a soft, overcast sky. Blurred trees and distant structures fade into the background, creating a dreamy, almost magical atmosphere, as if nature has wrapped itself in a gentle mist.  

As for our furry friend? Meet a **cheerful brown dog** who’s turned this damp day into a splash-tastic adventure! It’s wading through the shallow water, paws dipping in and out to create tiny, sparkling ripples that spread like laughter across the surface. With its tongue lolling out in a goofy, happy grin and tail wagging so fast it’s a blur, the pup looks like it’s saying, *“Rain? Pfft—who needs sunshine when you’ve got a whole pond to explore?!”*  

Every step it takes sends ripples dancing, as if the water itself is joining in the fun. It’s not just walking—it’s *strutting*, like a king or queen of the wet world, turning a gloomy day into a cel